# Pips and Connection 

In [18]:
%pip install mongoengine
%pip install pycoshark
%pip install matplotlib
%pip install fpdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

     --------------------------------------- 11.0/11.0 MB 19.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.

  Using cached seaborn-0.12.1-py3-none-any.whl (288 kB)


In [2]:
import re
import matplotlib.pyplot as plt
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, FileAction, Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
from mongoengine.connection import disconnect

# You may have to update this dict to match your DB credentials
credentials = {'db_user': '',
               'db_password': '',
               'db_hostname': 'localhost',
               'db_port': 27017,
               'db_authentication_database': '',
               'db_ssl_enabled': False}

uri = create_mongodb_uri_string(**credentials)

# disconnect(alias='default')
connect('smartshark_2_2', host=uri, alias='default')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [3]:
projectsObject = Project.objects().all()

projects =[]

for p in projectsObject:
    if(p.name is not  projects ):
        projects.append(p.name)


In [11]:
issue_type = {}
issue_type_refactoring = {}
for p in projects:
    project = Project.objects(name=p).get()

    if(project is not None):
        

        vcs_system = VCSSystem.objects(project_id=project.id).get()

        num_commits = Commit.objects(vcs_system_id=vcs_system.id).count()

        try: 

            issue_tracker = IssueSystem.objects(project_id=project.id).get()

            for issue in Issue.objects(issue_system_id=issue_tracker.id):
                if issue.issue_type_verified is not None:
                    if(issue.issue_type_verified not in issue_type):
                        issue_type[issue.issue_type_verified] = 1
                        issue_type_refactoring[issue.issue_type_verified] = {}
                    else:
                        issue_type[issue.issue_type_verified] += 1
                    
                    for commit in Commit.objects(linked_issue_ids=issue.id):
                        miner = Refactoring.objects(commit_id=commit.id, detection_tool='rMiner')
                        if miner is not None:
                            for ref in miner:
                                
                                if ref.type not in issue_type_refactoring[issue.issue_type_verified]:
                                    issue_type_refactoring[issue.issue_type_verified][ref.type] = 1
                                else:
                                    issue_type_refactoring[issue.issue_type_verified][ref.type] += 1

        except Exception as e:
            print("No issue tracker for project " + p)
            print(e)
   
print(issue_type_refactoring)

No issue tracker for project maven
2 or more items returned, instead of 1
No issue tracker for project tapestry-5
2 or more items returned, instead of 1
No issue tracker for project james
2 or more items returned, instead of 1
{'documentation': {'extract_variable': 9, 'extract_method': 15, 'parametrize_variable': 2, 'rename_class': 3, 'change_variable_type': 161, 'extract_attribute': 26, 'rename_method': 5, 'change_return_type': 8, 'rename_variable': 25, 'rename_attribute': 2, 'inline_variable': 1, 'move_class': 40, 'rename_parameter': 13, 'pull_up_attribute': 2, 'pull_up_method': 5, 'extract_and_move_method': 5, 'inline_method': 2, 'extract_interface': 2, 'push_down_method': 2}, 'bug': {'change_variable_type': 2533, 'rename_variable': 1054, 'extract_superclass': 61, 'pull_up_method': 480, 'pull_up_attribute': 150, 'extract_method': 1337, 'extract_variable': 892, 'inline_method': 125, 'rename_parameter': 671, 'extract_attribute': 1166, 'rename_method': 827, 'rename_attribute': 263, 're

In [21]:
from fpdf import FPDF
import os
import sys

def dic2pdf(dic):
    #create a pdf file
    pdf = FPDF()
    #add a page
    pdf.add_page()
    #set style and size of font
    #that you want in the pdf
    pdf.set_font("Arial", size = 15)
    #inserting an image in the pdf file
    #pdf.image("image_name.jpg", w = 30, h = 30)
    #create a cell
    pdf.cell(200, 10, txt = "Dictionary", ln = 1, align = 'C')
    #add another cell
    pdf.cell(200, 10, txt = " ", ln = 2, align = 'C')
    #add another cell
    pdf.cell(200, 10, txt = " ", ln = 3, align = 'C')
    #inserting the texts in pdf
    for i in dic:
        pdf.cell(200, 10, txt = str(i) + ": " + str(dic[i]), ln = 4, align = 'C')
    #save the pdf with name .pdf
    pdf.output("dic.pdf")



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40705 sha256=1959fd4c4d39506fec934eefd2cf2405b1723d3ebb93a0b7485c3f891117076d
  Stored in directory: c:\users\kleitonewerton\appdata\local\pip\cache\wheels\e7\3f\19\97faa9b38eab58ba402cb09bf5a7adb689fffa71612c4b042f
Successfully built fpdf


In [22]:
dic2pdf(issue_type_refactoring)

# Analise